## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [1]:
! pip install pipenv --upgrade


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
! pipenv --version

pipenv, version 2023.10.3


> **Answer 1**: 2023.10.3

## Question 2

* Use Pipenv to install Scikit-Learn version 1.3.1
* What's the first hash for scikit-learn you get in Pipfile.lock?

> **Note**: you should create an empty folder for homework
and do it there. 


In [3]:
! pipenv install scikit-learn==1.3.1

Installing scikit-learn==1.3.1...
Resolving scikit-learn==1.3.1...
✔ Installation Succeeded
⠋ Installing scikit-learn...
Installing dependencies from Pipfile.lock (7bfb14)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [4]:
import json
with open('Pipfile.lock') as f:
    j = json.load(f)
j['default']['scikit-learn']['hashes'][0]

'sha256:0c275a06c5190c5ce00af0acbb61c06374087949f643ef32d355ece12c4db043'

> **Answer 2**: 0c275a06c5190c5ce00af0acbb61c06374087949f643ef32d355ece12c4db043

## Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

```python
features = ['job','duration', 'poutcome']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)
```

> **Note**: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

* [DictVectorizer](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2023/05-deployment/homework/dv.bin?raw=true)
* [LogisticRegression](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2023/05-deployment/homework/model1.bin?raw=true)

With `wget`:

```bash
PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin
```

In [5]:
%%bash

PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin


--2023-10-16 18:21:53--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 842 [application/octet-stream]
Saving to: ‘model1.bin.1’

     0K                                                       100%  154M=0s

2023-10-16 18:21:53 (154 MB/s) - ‘model1.bin.1’ saved [842/842]

--2023-10-16 18:21:53--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"job": "retired", "duration": 445, "poutcome": "success"}
```

What's the probability that this client will get a credit? 

* 0.162
* 0.392
* 0.652
* 0.902

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum model1.bin dv.bin
8ebfdf20010cfc7f545c43e3b52fc8a1  model1.bin
924b496a89148b422c74a62dbc92a4fb  dv.bin
```

In [6]:
import pickle

with open('model1.bin', 'rb') as f:
    model = pickle.load(f)
    
with open('dv.bin', 'rb') as f:
    dv = pickle.load(f)

In [7]:
X_test = dv.transform({"job": "retired", "duration": 445, "poutcome": "success"})

In [8]:
y_test = model.predict_proba(X_test)[0, 1]
round(y_test, 3)

0.902

> **Answer 3**: 0.902

## Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit?

* 0.140
* 0.440
* 0.645
* 0.845

In [9]:
! pipenv install flask gunicorn

Installing flask...
Resolving flask...
✔ Installation Succeeded
⠋ Installing flask...
Installing gunicorn...
Resolving gunicorn...
✔ Installation Succeeded
⠋ Installing gunicorn...
Installing dependencies from Pipfile.lock (7bfb14)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [10]:
# start the webservice with: python webservice.py
input('Start the webservice with `python webservice.py` and press enter:')

Start the webservice with `python webservice.py` and press enter:


''

In [11]:
import requests

url = 'http://localhost:9991/predict'
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
answer = requests.post(url, json=client).json()
round(answer['credit_probability'], 3)

0.14

> **Answer 4**: 0.140

## Docker

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.10.12-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.10.12-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.10.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.10.12-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.

In [12]:
! sudo apt-get install -y docker.io

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
docker.io is already the newest version (24.0.5-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 586 not upgraded.


## Question 5

Download the base image `svizor/zoomcamp-model:3.10.12-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 47 MB
* 147 MB
* 374 MB
* 574 MB

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [13]:
! docker pull svizor/zoomcamp-model:3.10.12-slim

3.10.12-slim: Pulling from svizor/zoomcamp-model
Digest: sha256:e8441100b9d8da56344f50c673eb2daded3c61ce9565e45c3592c02f34fb3149
Status: Image is up to date for svizor/zoomcamp-model:3.10.12-slim
docker.io/svizor/zoomcamp-model:3.10.12-slim


In [14]:
! docker images svizor/*

REPOSITORY              TAG            IMAGE ID       CREATED      SIZE
svizor/zoomcamp-model   3.10.12-slim   08266c8f0c4b   7 days ago   147MB


> **Answer 5**: 147MB

## Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```docker
FROM svizor/zoomcamp-model:3.10.12-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn 

After that, you can build your docker image.

In [15]:
! docker build --no-cache -t hw5 .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  77.82kB
Step 1/8 : FROM svizor/zoomcamp-model:3.10.12-slim
 ---> 08266c8f0c4b
Step 2/8 : WORKDIR /app
 ---> Running in b7fbf75dba88
Removing intermediate container b7fbf75dba88
 ---> 23bc4b66716a
Step 3/8 : COPY ["Pipfile", "Pipfile.lock", "webservice.py", "./"]
 ---> dd9aa1946a89
Step 4/8 : RUN     pip install pipenv &&     pipenv install --deploy --system
 ---> Running in 32cbbff6769b
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

## Question 6

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit now?

* 0.168
* 0.530
* 0.730
* 0.968

In [16]:
# start the webservice with: python webservice.py
input('Start the docker image with `docker run --rm -it -p 9991:9991 hw5` and press enter:')

Start the docker image with `docker run --rm -it -p 9991:9991 hw5` and press enter:


''

In [17]:
url = "http://localhost:9991/predict"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
answer = requests.post(url, json=client).json()
round(answer['credit_probability'], 3)

0.727

> **Answer 6**: 0.730